### Protótipo de ETL no contexto dos dados estatísticos da covid-19 nos continentes

In [1]:

import pandas as pd

df = pd.read_csv(r'../csv/covid_continents.csv', index_col=0)

dataframe = df.copy()

pd.set_option('display.max_columns', None)   
pd.set_option('display.max_rows', None)      
pd.set_option('display.width', 300)         
pd.set_option('display.max_colwidth', None) 
pd.set_option('display.float_format', '{:.2f}'.format)

dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6 entries, 0 to 5
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   updated                 6 non-null      int64  
 1   cases                   6 non-null      int64  
 2   todayCases              6 non-null      int64  
 3   deaths                  6 non-null      int64  
 4   todayDeaths             6 non-null      int64  
 5   recovered               6 non-null      int64  
 6   todayRecovered          6 non-null      int64  
 7   active                  6 non-null      int64  
 8   critical                6 non-null      int64  
 9   casesPerOneMillion      6 non-null      float64
 10  deathsPerOneMillion     6 non-null      float64
 11  tests                   6 non-null      int64  
 12  testsPerOneMillion      6 non-null      float64
 13  population              6 non-null      int64  
 14  continent               6 non-null      object 
 15

# Atividade Prática: Tratamento e Padronização de Dados COVID-19 por Continente

---

## 1️⃣ Tradução e compreensão das colunas (snake_case)

| Coluna original | Coluna tratada (snake_case) | Significado |
|-----------------|----------------------------|-------------|
| updated | ultima_atualizacao | Timestamp da última atualização (epoch em ms) |
| cases | casos_totais | Total de casos acumulados |
| todayCases | casos_hoje | Casos registrados no dia |
| deaths | obitos_totais | Total de óbitos acumulados |
| todayDeaths | obitos_hoje | Óbitos registrados no dia |
| recovered | recuperados_totais | Total de recuperados |
| todayRecovered | recuperados_hoje | Recuperados no dia |
| active | casos_ativos | Casos ativos |
| critical | casos_criticos | Casos críticos |
| casesPerOneMillion | casos_por_milhao_habitantes | Casos por milhão de habitantes |
| deathsPerOneMillion | obitos_por_milhao_habitantes | Óbitos por milhão de habitantes |
| tests | total_testes | Total de testes realizados |
| testsPerOneMillion | testes_por_milhao_habitantes | Testes por milhão de habitantes |
| population | populacao_total | População total do continente |
| continent | continente | Nome do continente |
| activePerOneMillion | casos_ativos_por_milhao_habitantes | Casos ativos por milhão |
| recoveredPerOneMillion | recuperados_por_milhao_habitantes | Recuperados por milhão |
| criticalPerOneMillion | criticos_por_milhao_habitantes | Casos críticos por milhão |
| continentInfo | continente_info | Informações geográficas do continente (lat/long) |
| countries | paises | Lista de países pertencentes ao continente |

---

## 2️⃣ Exploração inicial do dataset

- [ ] Verificar número de linhas e colunas
- [ ] Inspecionar os tipos de dados (`dtypes`)
- [ ] Verificar valores nulos

---

## 3️⃣ Coluna `ultima_atualizacao` (timestamp)

Representa a última atualização em **epoch (milissegundos)**.

- [ ] Confirmar que o valor tem 13 dígitos
- [ ] Converter para datetime (`pd.to_datetime(unit='ms', utc=True)`)

---

## 4️⃣ Colunas de contagem (inteiros)

Colunas que representam contadores absolutos:

`casos_totais, casos_hoje, obitos_totais, obitos_hoje, recuperados_totais, recuperados_hoje, casos_ativos, casos_criticos, total_testes, populacao_total`

Para cada coluna:

- [ ] Garantir tipo `int`
- [ ] Substituir valores faltantes ou inconsistentes por 0
- [ ] Substituir valores negativos por 0
- [ ] Validar regras semânticas:
  - `casos_totais >= obitos_totais`
  - `casos_totais >= recuperados_totais`
  - `populacao_total > 0`

---

## 5️⃣ Colunas de indicadores por milhão (float)

Colunas derivadas:

`casos_por_milhao_habitantes, obitos_por_milhao_habitantes, testes_por_milhao_habitantes, casos_ativos_por_milhao_habitantes, recuperados_por_milhao_habitantes, criticos_por_milhao_habitantes`

Para cada coluna:

- [ ] Garantir tipo `float`
- [ ] Substituir valores faltantes ou inconsistentes
- [ ] Substituir valores negativos por 0
- [ ] Validar se os indicadores condizem com os contadores absolutos
- [ ] Recalcular indicadores por milhão (opcional):
  - `casos_por_milhao_habitantes = casos_totais / (populacao_total / 1_000_000)`
  - `obitos_por_milhao_habitantes = obitos_totais / (populacao_total / 1_000_000)`
  - `testes_por_milhao_habitantes = total_testes / (populacao_total / 1_000_000)`
  - `casos_ativos_por_milhao_habitantes = casos_ativos / (populacao_total / 1_000_000)`
  - `recuperados_por_milhao_habitantes = recuperados_totais / (populacao_total / 1_000_000)`
  - `criticos_por_milhao_habitantes = casos_criticos / (populacao_total / 1_000_000)`

---

## 6️⃣ Coluna `continente`

- [ ] Garantir tipo `string`
- [ ] Padronizar nomes (ex: remover espaços, maiúsculas consistentes)
- [ ] Validar valores válidos (existência de 6 continentes + Oceania)

---

## 7️⃣ Coluna `continente_info` (geolocalização)

Contém um dicionário serializado como string com latitude e longitude.

- [ ] Converter string para dicionário Python (`ast.literal_eval`)
- [ ] Extrair latitude → `continente_lat`
- [ ] Extrair longitude → `continente_long`
- [ ] Remover coluna original `continente_info`

---

## 8️⃣ Coluna `paises` (lista de países)

Contém uma lista serializada de países do continente.

- [ ] Converter string para lista Python (`ast.literal_eval`)
- [ ] Explodir lista criando uma tabela auxiliar `continente–pais`
- [ ] Remover coluna `paises` do dataframe principal

---

## 9️⃣ Valores faltantes e inconsistentes

- [ ] Identificar colunas com valores nulos
- [ ] Definir estratégia por coluna:
  - [ ] Substituir por zero
  - [ ] Substituir por média ou mediana
  - [ ] Manter como nulo
- [ ] Validar impacto nos indicadores

---

## 🔟 Padronização final

- [ ] Renomear colunas para **snake_case**
- [ ] Reordenar colunas de forma lógica
- [ ] Garantir consistência de tipos e dados
- [ ] Validar que dataframe final tem **uma linha por continente**
- [ ] Salvar versão limpa para análise ou pipeline (CSV, DB, API)

---

## 11️⃣ Documentação

- [ ] Documentar decisões de tratamento
- [ ] Justificar remoção ou criação de colunas
- [ ] Diferenciar dados brutos de dados derivados
- [ ] Avaliar se dataset está pronto para consumo em BI ou API
